# Network

* Tree, every node connects up until root
* node:
  * ingress:
    * websocket and/or
    * espnet
  * outgress: (except root)
    * websocket
    * espnet
  
## qos:

* client -> server
  * BUFFER - at a minimum don't miss !update
  * qos=2 or 3
* server -> client
  * squash buffer when disconnected?
  * qos = 1 (discard everything if disconnected)
  * also discards ?act
* ?ack # -> !ack #

## server

* qos = 0 (best effort)
* on connect, close existing connection to client if any
* buffer messages
* on disconnect, abort and discard all messages

## client

* qos = 1 (at least once)
* ?ack # -> !ack #
  * receiver processes messages (no queue)
  * responds to ?ack
* sender buffers messages until !ack received

In [ ]:
from io import StringIO
import asyncio
import aiohttp
import json

class Bridge:

    def __init__(self, ws, *, max_delay=2, size_threshold=1000) -> None:
        self.ws = ws
        self.max_delay = max_delay
        self.size_threshold = size_threshold
        self.buffer = StringIO()
        self.lines = []
        asyncio.create_task(self.flush_task())

    async def emit(self, event: dict) -> None:
        json.dump(event, self.buffer)
        self.buffer.write("\n")
        if self.buffer.tell() > self.size_threshold:
            await self.flush()

    async def flush(self) -> None:
        data = self.buffer.getvalue()
        if data:
            self.buffer = StringIO()
            await self.ws.send_str(data)
        asyncio.create_task(self.flush_task())

    async def flush_task(self):
        if self.ws.closed:
            return
        await asyncio.sleep(self.max_delay)
        await self.flush()

    def __aiter__(self):
        return self
    
    async def __anext__(self):
        while not self.lines:
            # fetch next batch of data from websocket
            msg = await self.ws.receive()
            if msg.type != aiohttp.WSMsgType.TEXT:
                raise StopAsyncIteration
            self.lines = msg.data.split("\n")[:-1]
        return json.loads(self.lines.pop(0))

    


In [3]:
import asyncio
import aiohttp

from microdot import Microdot, abort
from microdot.websocket import WebSocket, WebSocketError, websocket_upgrade

app = Microdot()

WebSocket.max_message_length = 256 * 1024

@app.route('/')
async def index(request):
    return 'Hello leaf!'

@app.route('/ping')
async def ping(request):
    return 'pong'

@app.route('/echo')
async def echo(request):
    # authenticate
    token = request.cookies.get('leaf-token')
    if token is None:
        abort(401)
    # open websocket
    ws = await websocket_upgrade(request)
    while True:
        try:
            message = await ws.receive()
            await ws.send(message, opcode=WebSocket.TEXT)
        except WebSocketError as e:
            print("microdot ws closed", type(e), e, "closed=", ws.closed)
            break
        except Exception as e:
            print("microdot ws error", type(e), e, ws.closed)
            break

asyncio.create_task(app.start_server(host='localhost', port=5000, debug=False, ssl=None))

ws_url = "http://localhost:5000/echo"

async def receiver_task(ws):
    try:
        async for msg in ws:
            print("GOT echo", msg)
    except aiohttp.WebSocketError as e:
        print("aiohttp.ws", type(e), e)
        return

cookies = {'leaf-token': 'some token'}

async with aiohttp.ClientSession(cookies=cookies) as session:
    try:
        async with session.ws_connect(ws_url) as ws:
            print("client CONNECTED")
            asyncio.create_task(receiver_task(ws))
            await asyncio.sleep(1)
            for i in range(10):
                await ws.send_str(f"Hello, world! {i}")
                await asyncio.sleep(0)
    except aiohttp.WSServerHandshakeError as e:
        print("client WSServerHandshakeError", type(e), e)

client CONNECTED
GOT echo WSMessage(type=<WSMsgType.TEXT: 1>, data='Hello, world! 0', extra='')
GOT echo WSMessage(type=<WSMsgType.TEXT: 1>, data='Hello, world! 1', extra='')
GOT echo WSMessage(type=<WSMsgType.TEXT: 1>, data='Hello, world! 2', extra='')
GOT echo WSMessage(type=<WSMsgType.TEXT: 1>, data='Hello, world! 3', extra='')
GOT echo WSMessage(type=<WSMsgType.TEXT: 1>, data='Hello, world! 4', extra='')
GOT echo WSMessage(type=<WSMsgType.TEXT: 1>, data='Hello, world! 5', extra='')
GOT echo WSMessage(type=<WSMsgType.TEXT: 1>, data='Hello, world! 6', extra='')
